In [1]:
import numpy as np
import matplotlib.pyplot as pl
from sklearn.utils import class_weight
import tensorflow as tf
from tensorflow.keras import Sequential, activations, Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    BatchNormalization, Activation, SpatialDropout2D, GlobalAveragePooling2D,
    Dense, Dropout, Flatten,
    Conv2D, MaxPooling2D, SeparableConv2D
)
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

# Import project utils scripts
import os
import sys

src_path = os.path.join('../src/')

if src_path not in sys.path:
    sys.path.append(src_path)

from statisticsUtils import recall_precision_fscore_from_confusion_matrix
from labelsUtils import Label
from regionUtils import vietnam_labels_coordinates
from rasterUtils import make_dataset_from_raster_files
from convNetUtils import cross_validation, evaluate_model
from visualizationUtils import label_first_detections, plot_confusion_matrix
from bandUtils import Band
from config import *

/home/simon/miniconda3/envs/tb/lib/python3.9/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
NB_TESTS = 4
NB_PIXEL_AROUND = 4
EPOCHS = 2000
LABELS_COORDINATES = vietnam_labels_coordinates()

In [3]:
# Without :
# Label.INTERCROP,
# Label.STICK_FOR_PEPPER,   
# Label.PEPPER_AND_COFFEE,     
# Label.PEPPER_AND_OTHER
# Label.SPARE_TREE,     


bands = [
    Band.COASTAL_AEROSOL.value, 
    Band.BLUE.value, 
    Band.GREEN.value, 
    Band.RED.value, 
    Band.NIR.value, 
    Band.SWIR1.value, 
    Band.SWIR2.value, 
]

labels = [
    Label.COFFEE,
    Label.DENSE_FOREST,
    Label.RUBBER,
    Label.SEASONAL_AGRICULTURE,
    Label.URBAN,
    Label.WATER,
    Label.OTHER_TREE,
    Label.NATIVE_NO_TREE,
    Label.PEPPER,
    Label.TEA,
    Label.RICE,     
    Label.DECIDUOUS_FOREST,
    Label.PINE_TREES,     
    Label.SHRUBLAND_BUSHLAND,     
    Label.GRASSLAND,     
    Label.SECONDARY_DEGRADED_FOREST,     
    Label.MINE_BARESOIL,
]

dataset_args = dict(
    labels = labels,
    raster_paths = [os.path.join(DATA_ROOT_PATH, 'Vietnam_2018_january_to_april_collection2/merged.tif')],
    labels_coordinates_list = [LABELS_COORDINATES],
    nb_pixel_around = NB_PIXEL_AROUND
)

dataset = make_dataset_from_raster_files(**dataset_args)

In [ ]:
model_name = "january_to_april_2018_every_usable_labels"

# images have all the same shapes, take the shape of the first image
image_width = len(dataset[0][1][0][0])
image_height = len(dataset[0][1][0])
image_depth = len(bands)
nb_outputs = len(labels)

# Create model
model = Sequential([
    Rescaling(0.0000275, offset=0.2, input_shape=(image_width, image_height, image_depth)),
    BatchNormalization(),
    Conv2D(filters=32, kernel_size=(3, 3), padding="same", activation="relu"),
    Conv2D(filters=32, kernel_size=(3, 3), padding="same", activation="relu"),
    MaxPooling2D(pool_size=(3, 3)),
    Conv2D(filters=64, kernel_size=(3, 3), padding="same", activation="relu"),
    Conv2D(filters=64, kernel_size=(3, 3), padding="same", activation="relu"),
    MaxPooling2D(pool_size=(3, 3)),
    SpatialDropout2D(0.25),
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.25),
    Dense(nb_outputs, activation='softmax'),
])

mean_loss, mean_accuracy, histories, conf_matrix = cross_validation(model, dataset, bands, labels, EPOCHS, NB_TESTS, early_stopping=True, with_model_checkpoint=True, model_name=model_name)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling (Rescaling)        (None, 9, 9, 7)           0         
_________________________________________________________________
batch_normalization (BatchNo (None, 9, 9, 7)           28        
_________________________________________________________________
conv2d (Conv2D)              (None, 9, 9, 32)          2048      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 9, 9, 32)          9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 3, 3, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          3

351/351 [==============================] - 1s 3ms/step - loss: 1.1152 - accuracy: 0.5029 - val_loss: 1.4864 - val_accuracy: 0.4959
Epoch 41/2000
351/351 [==============================] - 1s 3ms/step - loss: 1.2542 - accuracy: 0.4742 - val_loss: 1.3668 - val_accuracy: 0.5339
Epoch 42/2000
351/351 [==============================] - 1s 3ms/step - loss: 1.2146 - accuracy: 0.4766 - val_loss: 1.5816 - val_accuracy: 0.4519
Epoch 43/2000
351/351 [==============================] - 1s 3ms/step - loss: 1.1539 - accuracy: 0.4958 - val_loss: 1.3101 - val_accuracy: 0.5385
Epoch 44/2000
351/351 [==============================] - 1s 3ms/step - loss: 1.0868 - accuracy: 0.5078 - val_loss: 1.3214 - val_accuracy: 0.5492
Epoch 45/2000
351/351 [==============================] - 1s 3ms/step - loss: 1.3625 - accuracy: 0.4809 - val_loss: 1.4926 - val_accuracy: 0.4675
Epoch 46/2000
351/351 [==============================] - 1s 3ms/step - loss: 1.1459 - accuracy: 0.4936 - val_loss: 1.4206 - val_accuracy: 0.5385

351/351 [==============================] - 1s 4ms/step - loss: 0.9575 - accuracy: 0.5552 - val_loss: 1.1508 - val_accuracy: 0.6146
Epoch 97/2000
351/351 [==============================] - 1s 3ms/step - loss: 0.9232 - accuracy: 0.5623 - val_loss: 1.2222 - val_accuracy: 0.5922
Epoch 98/2000
351/351 [==============================] - 1s 3ms/step - loss: 1.1993 - accuracy: 0.5353 - val_loss: 1.2361 - val_accuracy: 0.5861
Epoch 99/2000
351/351 [==============================] - 1s 3ms/step - loss: 1.0263 - accuracy: 0.5407 - val_loss: 1.1945 - val_accuracy: 0.5872
Epoch 100/2000
351/351 [==============================] - 1s 3ms/step - loss: 1.1593 - accuracy: 0.5272 - val_loss: 1.1491 - val_accuracy: 0.6014
Epoch 101/2000
351/351 [==============================] - 1s 3ms/step - loss: 1.2430 - accuracy: 0.5449 - val_loss: 1.2996 - val_accuracy: 0.5641
Epoch 102/2000
351/351 [==============================] - 1s 3ms/step - loss: 1.0707 - accuracy: 0.5685 - val_loss: 1.1206 - val_accuracy: 0.6

In [ ]:
for i, history in enumerate(histories):
    pl.plot(history.history['loss'], label='Training', color="blue")
    pl.plot(history.history['val_loss'], label='Testing', color="orange")
    
    if i == 0:
        pl.legend()
        
    pl.grid()

In [ ]:
for i, history in enumerate(histories):
    pl.plot(history.history['f1_score_train'], label='Training', color="blue")
    pl.plot(history.history['f1_score_val'], label='Testing', color="orange")
    
    if i == 0:
        pl.legend()
        
    pl.grid()

In [ ]:
pl.figure(figsize=(12, 10), dpi=80)

plot_confusion_matrix(conf_matrix, labels)

In [ ]:
recall, precision, fscore = recall_precision_fscore_from_confusion_matrix(conf_matrix)

print("Precisions: ", precision)
print("Mean precision: ", np.mean(precision))
print("\nRecalls: ", recall)
print("Mean recall: ", np.mean(recall))
print("\nF-Score: ", fscore)
print("Mean f-score: ", np.mean(fscore))